## 미세먼지 예측(LSTM)

In [71]:
# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# 데이터 불러오기
data_path = 'dataset/seoul_pm10.csv'  # 데이터 경로
df = pd.read_csv(data_path, encoding='cp949')

# 데이터 확인
print(df.head())
#print(df.isnull().sum())
print(df.shape)
#dateList = list(df['date'].unique())
#dateList.sort()
#print(len(dateList))
#print(227760/8760) # 각 시간(8760개)마다 26개의 지역이 반복됨
df_List = df.values

df_gangnam = []
for d in df_List:
    if d[1]=='강남구':
        df_gangnam.append(d)
print(len(df_gangnam))

df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df.dropna(inplace=True)  # 결측치 제거


               date area  pm10  pm2.5
0  2022-12-31 23:00  강남구  57.0   44.0
1  2022-12-31 23:00  강동구  68.0   55.0
2  2022-12-31 23:00  강북구  59.0   42.0
3  2022-12-31 23:00  강서구  62.0   40.0
4  2022-12-31 23:00  관악구  57.0   38.0
(227759, 4)
8760


In [ ]:
# 2. 데이터 전처리
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df)

def create_dataset(dataset, look_back=26):
    X, y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:i + look_back, 0])
        y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(y)

look_back = 26  # 26개의 지역씩 나눔
X, y = create_dataset(scaled_data, look_back)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))  # LSTM 입력 형태